In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
get_ipython().run_line_magic('matplotlib', 'notebook')
%matplotlib notebook
# interactive plots, must add before each plot or they will all add to same plot

In [ ]:
# MidEast January
# with xr.open_dataset('../Desktop/MidEastJan.nc') as ds:  (on Mac)
with xr.open_dataset('../Oct30/Jan/MidEast/MidEastJan.nc') as ds:
    print(ds)

In [ ]:
with xr.open_dataset('../Oct30/Jan/MidEast/MidEastJan.nc') as ds:
    fig, (ax1,ax2,ax3) = plt.subplots(3, 1, sharex=True, figsize=(9,10))
    fig.subplots_adjust(hspace = 0.1)
    
    ax1.set_title('MidEast ADCP 150 kHz - January 2018 - u, v, w velocities')
    ax1.set_ylabel('Depth [m]')
    im1 = ax1.pcolormesh(np.linspace(0,30,720), -ds.depth, ds.u.T, rasterized=True, cmap='bwr')
    cbar1 = fig.colorbar(im1, ax=ax1, fraction=0.05, pad=0.02, aspect=40)
    cbar1.set_label('Velocity [m/s]')
    
    im2 = ax2.pcolormesh(np.linspace(0,30,720), -ds.depth, ds.v.T, rasterized=True, cmap='bwr')
    cbar2 = fig.colorbar(im2, ax=ax2, fraction=0.05, pad=0.02, aspect=40)
    
    ax3.set_xlabel('Time [days]')
    im3 = ax3.pcolormesh(np.linspace(0,30,720), -ds.depth, ds.w.T, rasterized=True, cmap='bwr')
    cbar3 = fig.colorbar(im3, ax=ax3, fraction=0.05, pad=0.02, aspect=40)

In [ ]:
# Axis January
with xr.open_dataset('../Oct30/Jan/Axis/AxisJan.nc') as ds:
    print(ds)

In [ ]:
with xr.open_dataset('../Oct30/Jan/Axis/AxisJan.nc') as ds:
    fig, (ax1,ax2,ax3) = plt.subplots(3, 1, sharex=True, figsize=(9,10))
    fig.subplots_adjust(hspace = 0.1)
    
    ax1.set_title('Axis ADCP 75 kHz - January 2018 - u, v, w velocities')
    ax1.set_ylabel('Depth [m]')
    im1 = ax1.pcolormesh(np.linspace(0,30,720), -ds.depth, ds.u.T, rasterized=True, cmap='bwr')
    cbar1 = fig.colorbar(im1, ax=ax1, fraction=0.05, pad=0.02, aspect=40)
    cbar1.set_label('Velocity [m/s]')
    
    im2 = ax2.pcolormesh(np.linspace(0,30,720), -ds.depth, ds.v.T, rasterized=True, cmap='bwr')
    cbar2 = fig.colorbar(im2, ax=ax2, fraction=0.05, pad=0.02, aspect=40)
    
    ax3.set_xlabel('Time [days]')
    im3 = ax3.pcolormesh(np.linspace(0,30,720), -ds.depth, ds.w.T, rasterized=True, cmap='bwr')
    cbar3 = fig.colorbar(im3, ax=ax3, fraction=0.05, pad=0.02, aspect=40)

In [ ]:
# Upper Slope January
with xr.open_dataset('../Oct30/Jan/UpperSlope/UpperSlopeJan.nc') as ds:
    print(ds)

In [ ]:
with xr.open_dataset('../Oct30/Jan/UpperSlope/UpperSlopeJan.nc') as ds:
    fig, (ax1,ax2,ax3) = plt.subplots(3, 1, sharex=True, figsize=(9,10))
    fig.subplots_adjust(hspace = 0.1)
    
    ax1.set_title('Upper Slope ADCP 75 kHz - January 2018 - u, v, w velocities')
    ax1.set_ylabel('Depth [m]')
    im1 = ax1.pcolormesh(np.linspace(0,30,720), -ds.depth, ds.u.T, rasterized=True, cmap='bwr')
    cbar1 = fig.colorbar(im1, ax=ax1, fraction=0.05, pad=0.02, aspect=40)
    cbar1.set_label('Velocity [m/s]')
    
    im2 = ax2.pcolormesh(np.linspace(0,30,720), -ds.depth, ds.v.T, rasterized=True, cmap='bwr')
    cbar2 = fig.colorbar(im2, ax=ax2, fraction=0.05, pad=0.02, aspect=40)
    
    ax3.set_xlabel('Time [days]')
    im3 = ax3.pcolormesh(np.linspace(0,30,720), -ds.depth, ds.w.T, rasterized=True, cmap='bwr')
    cbar3 = fig.colorbar(im3, ax=ax3, fraction=0.05, pad=0.02, aspect=40)

In [ ]:
# MidEast July
with xr.open_dataset('../Oct30/Jul/MidEast/MidEastJul.nc') as ds:
    print(ds)

In [ ]:
with xr.open_dataset('../Oct30/Jul/MidEast/MidEastJul.nc') as ds:
    fig, (ax1,ax2,ax3) = plt.subplots(3, 1, sharex=True, figsize=(9,10))
    fig.subplots_adjust(hspace = 0.1)
    
    ax1.set_title('MidEast ADCP 150 kHz - July 2018 - u, v, w velocities')
    ax1.set_ylabel('Depth [m]')
    im1 = ax1.pcolormesh(np.linspace(0,30,720), -ds.depth, ds.u.T, rasterized=True, cmap='bwr')
    cbar1 = fig.colorbar(im1, ax=ax1, fraction=0.05, pad=0.02, aspect=40)
    cbar1.set_label('Velocity [m/s]')
    
    im2 = ax2.pcolormesh(np.linspace(0,30,720), -ds.depth, ds.v.T, rasterized=True, cmap='bwr')
    cbar2 = fig.colorbar(im2, ax=ax2, fraction=0.05, pad=0.02, aspect=40)
    
    ax3.set_xlabel('Time [days]')
    im3 = ax3.pcolormesh(np.linspace(0,30,720), -ds.depth, ds.w.T, rasterized=True, cmap='bwr')
    cbar3 = fig.colorbar(im3, ax=ax3, fraction=0.05, pad=0.02, aspect=40)

In [ ]:
# Upper Slope July
with xr.open_dataset('../Oct30/Jul/UpperSlope/UpperSlopeJul.nc') as ds:
    print(ds)

In [ ]:
with xr.open_dataset('../Oct30/Jul/UpperSlope/UpperSlopeJul.nc') as ds:
    fig, (ax1,ax2,ax3) = plt.subplots(3, 1, sharex=True, figsize=(9,10))
    fig.subplots_adjust(hspace = 0.1)
    
    ax1.set_title('Upper Slope ADCP 75 kHz - July 2018 - u, v, w velocities')
    ax1.set_ylabel('Depth [m]')
    im1 = ax1.pcolormesh(np.linspace(0,30,720), -ds.depth, ds.u.T, rasterized=True, cmap='bwr')
    cbar1 = fig.colorbar(im1, ax=ax1, fraction=0.05, pad=0.02, aspect=40)
    cbar1.set_label('Velocity [m/s]')
    
    im2 = ax2.pcolormesh(np.linspace(0,30,720), -ds.depth, ds.v.T, rasterized=True, cmap='bwr')
    cbar2 = fig.colorbar(im2, ax=ax2, fraction=0.05, pad=0.02, aspect=40)
    
    ax3.set_xlabel('Time [days]')
    im3 = ax3.pcolormesh(np.linspace(0,30,720), -ds.depth, ds.w.T, rasterized=True, cmap='bwr')
    cbar3 = fig.colorbar(im3, ax=ax3, fraction=0.05, pad=0.02, aspect=40)